In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class SSM(nn.Module):
    def __init__(self, d_inner, state_size, device='cuda'):
        """
        SSM 레이어 초기화 (논문 수식 기반)

        Args:
            d_inner (int): 내부 차원 크기 (D)
            state_size (int): 상태 공간의 크기 (N)
            device (str): 모델 파라미터를 로드할 장치
        """
        super(SSM, self).__init__()
        self.d_inner = d_inner
        self.state_size = state_size
        self.device = device

        # 입력 x -> Δ, B, C 계산용 프로젝션 정의
        dt_rank = math.ceil(d_inner / 16)
        self.x_proj = nn.Linear(d_inner, dt_rank + state_size * 2, bias=False, device=device)
        self.dt_proj = nn.Linear(dt_rank, d_inner, bias=True, device=device)

        # 연속 시간 상태 행렬 A (A_log) 정의 - 학습 가능 파라미터
        # 논문 Eq (1a)의 A에 해당 (실제로는 이산화에 사용됨)
        A = torch.arange(1, state_size + 1, dtype=torch.float32, device=device).repeat(d_inner, 1)
        self.A_log = nn.Parameter(torch.log(A)) # shape: (d_inner, N)

        # 피드스루 D 정의 - 학습 가능 파라미터
        # Mamba의 출력 수식 y_t = C_t h_t + D x_t 에 사용됨
        self.D = nn.Parameter(torch.ones(d_inner, device=device)) # shape: (d_inner,)

        print(f"SSM Layer Initialized: d_inner={d_inner}, state_size={state_size}, dt_rank={dt_rank}")
        
    # 단계 2: 이산화 (Discretization) - 논문 Eq (4) 구현
    def discretization(self, delta, B):
        """
        연속 시간 파라미터(A, B)와 시간 스텝(delta)을 사용하여
        이산 시간 파라미터(Ā, B̄)를 계산합니다. (ZOH 방식 근사 - Eq 4)

        Args:
            delta (torch.Tensor): 시간 스텝 Δ. shape: [B, L, d_inner]
            B (torch.Tensor): 연속 시간 입력 행렬 B (입력 의존적). shape: [B, L, state_size]

        Returns:
            Tuple[torch.Tensor, torch.Tensor]:
                - delta_A (torch.Tensor): 이산화된 상태 전이 행렬 Ā. shape: [B, L, d_inner, state_size]
                - delta_B (torch.Tensor): 이산화된 입력 행렬 B̄. shape: [B, L, d_inner, state_size]
        """
        # 연속 시간 파라미터 A 계산
        A = -torch.exp(self.A_log.float()) # shape: (d_inner, state_size)

        # --- Ā = exp(ΔA) 계산 --- (Eq 4 첫 부분)
        # Broadcasting 사용: delta (B, L, D, 1) * A (1, 1, D, N) -> (B, L, D, N)
        dA = delta.unsqueeze(-1) * A.unsqueeze(0).unsqueeze(0) # ΔA 계산
        delta_A = torch.exp(dA) # shape: (B, L, D, N)

        # --- B̄ = (ΔA)⁻¹ (exp(ΔA) - I) B 계산 --- (Eq 4 두 번째 부분, B 사용 버전)
        # (exp(ΔA) - 1) 계산
        delta_A_minus_1 = delta_A - 1.0 # exp(ΔA) - I 부분

        # (ΔA)⁻¹ 부분 계산 (A가 0에 가까울 때 수치 안정성 중요)
        # 여기서는 간단히 분모에 작은 값(1e-10)을 더하여 0 나누기 방지
        dA_inv = 1.0 / (dA + 1e-10)

        # B̄ 계산
        delta_B = dA_inv * delta_A_minus_1 * B.unsqueeze(2) # shape: (B, L, D, N)

        return delta_A, delta_B
    
    # 단계 3: 순전파 (Forward Pass) - 논문 Eq (2a), (2b) + Mamba 특징 적용
    def forward(self, x):
        """ SSM 레이어 순전파 연산 수행 """
        B, L, D = x.shape
        N = self.state_size

        # 3-1. 입력 x로부터 Δ, B, C 동적 계산
        x_proj_out = self.x_proj(x) # (B, L, dt_rank + 2*N)
        dt_inter, B_ssm, C_ssm = torch.split(
            x_proj_out, [self.dt_proj.in_features, N, N], dim=-1
        )
        # B_ssm: 연속 시간 B (입력 의존적), shape: (B, L, N)
        # C_ssm: 이산 시간 C (입력 의존적), shape: (B, L, N) - Eq (2b)의 C 역할

        dt = self.dt_proj(dt_inter) # (B, L, D)
        delta = F.softplus(dt)      # Δ 계산, shape: (B, L, D)

        # 3-2. 이산화 (Discretization) - Eq (4) 호출
        delta_A, delta_B = self.discretization(delta, B_ssm) # Ā, B̄ 계산
        # delta_A (Ā): (B, L, D, N), delta_B (B̄): (B, L, D, N)

        # 3-3. Scan 연산 (상태 h 계산) - Eq (2a) 구현
        delta_B_u = delta_B * x.unsqueeze(-1) # 입력 항 B̄*x 계산, shape: (B, L, D, N)

        h = torch.zeros(B, L, D, N, device=x.device, dtype=x.dtype) # 상태 h 저장 공간
        h_prev = torch.zeros(B, D, N, device=x.device, dtype=x.dtype) # 초기 상태 h_0 = 0
        for t in range(L):
            # 상태 업데이트: h_t = Ā_t * h_{t-1} + B̄_t * x_t (Eq 2a)
            # delta_A[:, t] 는 t 시점의 Ā_t 역할
            h_t = delta_A[:, t] * h_prev + delta_B_u[:, t] # shape: (B, D, N)
            h[:, t] = h_t
            h_prev = h_t

        # 3-4. 출력 계산 - Eq (2b) 기반 + Mamba 특징 (C_t, D)
        # y_t = C_t * h_t + D * x_t
        y_state_contribution = torch.einsum('bln,bldn->bld', C_ssm, h) # C_t * h_t 부분
        y = y_state_contribution + x * self.D # + D * x_t 부분

        return y

In [2]:
B = 4  # 배치 크기
L = 512 # 시퀀스 길이 (순차 루프 때문에 너무 길면 매우 느려짐)
D = 128 # 내부 차원 (d_inner)
N = 64  # 상태 공간 크기 (state_size)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [3]:
ssm_model = SSM(d_inner=D, state_size=N, device=device)

SSM Layer Initialized: d_inner=128, state_size=64, dt_rank=8


In [4]:
x_sample = torch.randn(B, L, D, device=device)
print(f"Input shape (x): {x_sample.shape}")


y_output = ssm_model(x_sample)
print("\nSample output values (first batch, first sequence element):")
print(y_output[0, 0, :10]) # 첫 번째 배치, 첫 번째 시퀀스 요소의 앞 10개 값 출력

Input shape (x): torch.Size([4, 512, 128])

Sample output values (first batch, first sequence element):
tensor([ 0.4276, -0.3772, -0.4217,  0.0248, -0.2643,  0.2048, -0.6259,  0.6269,
        -0.1294, -0.3750], device='cuda:0', grad_fn=<SliceBackward0>)
